# CNN Convolutional Neural Network
## Curso 2 Udemy

## Actualizar a Python 3.10

In [ ]:
!sudo apt-get install python3.10
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10
!sudo update-alternatives --config python3
!sudo apt install python3-pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10-minimal
Suggested packages:
  python3.10-venv binfmt-support
The following NEW packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10 python3.10-minimal
0 upgraded, 4 newly installed, 0 to remove and 20 not upgraded.
Need to get 5,097 kB of archives.
After this operation, 19.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.10-minimal amd64 3.10.7-1+bionic1 [823 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.10-minimal amd64 3.10.7-1+bionic1 [1,970 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bion

In [1]:
!python3 --version

Python 3.7.14


## Librerias

In [2]:
import tensorflow as tf
tf.keras.backend.clear_session()
print(tf.__version__)

2.8.2


In [3]:
import numpy as np
import matplotlib as plt
import pandas as pd

In [4]:
import sklearn 
# import theano

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!ls

cnn.ipynb  dataset


## Inicializar la CNN

In [13]:
classifier = tf.keras.models.Sequential()

# Construir una CNN
## Convolucion --> Max Pooling --> Flattening --> Full Connection

### Convolution
Aplicar filtros para crear el mapa de caracteristicas

In [14]:
# filters=32 : filtros (mapas de caracteristicas) para cada imagen de entrada.
# kernel_size=(3, 3) : Convolucion con matriz de 3x3.
# input_shape (64, 64, 3) : vector bidimensional de entrada de 64x64, en un vector de 3 dimesiones (3 filtros de colores).
classifier.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), input_shape=(64, 64, 3), activation='relu'))

### Max Polling
Reducir la dimension de las imagenes 'filtros'.

Crear un nuevo mapa de caracteristicas mas pequeños.

In [15]:
# pool_size=(2, 2) : definir la matriz de convolucion. 
# --> En este caso esta matriz de 2x2 reduce un 75% el tamaño de la imagen de entrada.
# --> Esta matriz escoje el pixel de mayor valor/intensidad y se queda solo con ese.
# strides : saltos de la matriz hacia el lado para convolucionar.
# --> Por defecto toma el mismo valor del pool_size, en este caso : 2.
classifier.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

### Flattening
Convertir/aplanar los outputs (mapas de caracteristicas) a un vector unidimencional.

In [16]:
classifier.add(tf.keras.layers.Flatten())

## Full Connection
Ingestar el 'Flattening' (vector unidimencional de cada mapa de caracteristica) a una ANN.

Crear la ANN.

In [17]:
# Hidden Layer One
classifier.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Hidden Layer Two
classifier.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Compilar la CNN

In [18]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Ajuste de las imagenes para la CNN. 
Ajustar las imagenes de entrada a una dimension de 64x64 desde un directorio.

Aplicar transformaciones a las imagenes reescalandolas, volteandolas, zoom, ... para aumentar el DataSet.

In [19]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                shear_range=0.2,
                                                                zoom_range=0.2,
                                                                horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory('dataset/training_set',
                                                      target_size=(64, 64),
                                                      batch_size=32,
                                                      class_mode='binary')

testing_dataset = test_datagen.flow_from_directory('dataset/test_set',
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Entrenar la CNN.

Tarda arto, un par de horas, por eso no lo he completado la ejecucion.

In [23]:
classifier.fit(training_dataset,
                        steps_per_epoch=8000,
                        epochs=5,
                        validation_data=testing_dataset,
                        validation_steps=2000)

Epoch 1/5
  35/8000 [..............................] - ETA: 23:11:22 - loss: 0.8815 - accuracy: 0.5170

KeyboardInterrupt: ignored

## Hacer una prediccion
Ingesta de una foto mia de mi gato o perro.

In [24]:
# target-size=(64, 64) : re ajusta la imagen a una dimension de 64x64 pixeles bidimencional.
# --> (64, 64)
image_pred = tf.keras.preprocessing.image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64, 64))
# Transformar la imagen a un array de 3 dimensiones.
# --> (64, 64, 3)
image_pred = tf.keras.preprocessing.image.img_to_array(image_pred)
# transformar la imagen a un array de 4 dimensiones.
# --> (1, 64, 64, 3)
image_pred = np.expand_dims(image_pred, axis=0)

Hacer prediccion

In [27]:
# pred : se setea a valor 0 o 1 de acuerdo a la clase que ha predecido (gato o perro)
pred = classifier.predict(image_pred)
print(pred[0][0])
# Saber si es perro o gato por el Indice.
# Estraemos los indices de las clases del modelo ya entrenado para saber que animal
# es 0 y 1.
training_dataset.class_indices

1.0


{'cats': 0, 'dogs': 1}